In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import tensorflow as tf
import tensorflowjs as tfjs
import numpy as np
import os, time, json, dill, textwrap, re, string, shutil
from tqdm import tqdm
from MyModel import MyModel, OneStep
from tokenizer import tokenizer, detokenizer

In [4]:
policy = tf.keras.mixed_precision.Policy('float32')
tf.keras.mixed_precision.set_global_policy(policy)

print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Compute dtype: float32
Variable dtype: float32


In [5]:
# gpu growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
# load model
model_name = "models/model1-test2"
meta = json.load(open(f"{model_name}/meta.json"))
vocab = json.load(open(f"{model_name}/vocab.json"))

In [7]:
model = MyModel(vocab_size=meta["vocab_size"], embedding_dim=meta["embedding_dim"], rnn_units=meta["rnn_units"])

one_step_model = OneStep(model)
model(tf.zeros([1, 1]))

OneStep.__init__


<tf.Tensor: shape=(1, 1, 5000), dtype=float32, numpy=
array([[[-0.00034489,  0.00017292,  0.00110298, ..., -0.00138007,
         -0.00050311,  0.00085747]]], dtype=float32)>

In [8]:
model.load_weights(f"{model_name}/weights.h5")

In [9]:
def generate_text(starting, num_generate, onestepmodel=None):
    if onestepmodel is None:
        onestepmodel = one_step_model
    states = None
    # next_char = text_vectorization(tokenizer(starting))
    next_char = tf.constant(
        [(vocab.index(c) if c in vocab else vocab.index("[UNK]")) for c in tokenizer(starting)]
    )
    print(next_char)
    result = next_char.numpy()

    for _ in tqdm(range(num_generate)):
        next_char = tf.expand_dims(next_char, 0)
        next_char, states = one_step_model.generate_one_step(next_char, states=states, temperature=0.9)
        # while True:
        #     next_char1, states1 = one_step_model.generate_one_step(next_char, states=states, temperature=0.1)
        #     if not (next_char1[0] == 1):
        #         next_char, states = next_char1, states1
        #         break
        if next_char[0] == vocab.index("<SPECIAL:END>"):
            break

        result = np.append(result, next_char.numpy(), axis=0)
    result = detokenizer([vocab[r] for r in result])
    w = textwrap.TextWrapper(width=90, break_long_words=True, replace_whitespace=False)
    print("=" * 50)
    print(w.fill(result))


generate_text("I recently published my thesis for", 100)

tf.Tensor([  2   7 253 183  15 142  18], shape=(7,), dtype=int32)


100%|██████████| 100/100 [00:01<00:00, 52.77it/s]

I recently published my thesis for a year before. I don't know if I should mention my
paper in this paper before I handed it out to the authors. Would it be appropriate to
thank the editor based on the blog suggestion which I want to publish in the journal
because my paper is likely that he has 2 good reviews?

Anyway, it seems like time was
that he has contributed more to my work, and he took two of his data analysis papers. The
book is still widely in its


In [10]:
# save model as saved_model with signature

NEW_MODEL_NAME = "tmp/model2-test"

tf.saved_model.save(
    one_step_model,
    f"{NEW_MODEL_NAME}/saved_model",
    signatures=one_step_model.generate_one_step.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int32, name="input_ids"),
        tf.TensorSpec(shape=[2, None, None], dtype=tf.float32, name="states"),
        tf.TensorSpec(shape=[], dtype=tf.float32, name="temperature"),
    ),
)

one_step_model2 = tf.saved_model.load(f"{NEW_MODEL_NAME}/saved_model")


generate_text("I recently published my thesis for", 100, onestepmodel=one_step_model2)

INFO:tensorflow:Assets written to: tmp/model2-test/saved_model\assets


INFO:tensorflow:Assets written to: tmp/model2-test/saved_model\assets


tf.Tensor([  2   7 253 183  15 142  18], shape=(7,), dtype=int32)


 76%|███████▌  | 76/100 [00:00<00:00, 168.89it/s]

I recently published my thesis for a (new) Phd.

I would like to ask whether this is the
case, should I include a note that an author is he / she will be to contact the paper on
his / her Phd negatively or assume that I am the sole author of the paper, or would there
be any potential ethical in which a claim is not working on?


In [11]:
generate_text("My thesis is good, but", 100, onestepmodel=one_step_model2)

tf.Tensor([  2  15 142  14 104   5  28], shape=(7,), dtype=int32)


100%|██████████| 100/100 [00:00<00:00, 178.73it/s]

My thesis is good, but unfortunately I have a strong inclination to reject the paper
without being asked out.

For my first few days, my advisor wanted to take the draft of
the second paper online, but the people just found my thesis.

My Phd advisor told me to
not mention that I am on track committee (this process of mine).

I have been struggling
to contact all the talks from the world and physical (I see it as a very bad teacher)


In [16]:
tfjs.converters.convert_tf_saved_model(
    f"{NEW_MODEL_NAME}/saved_model", f"{NEW_MODEL_NAME}/tfjs", quantization_dtype_map={"float16": "*"}
)

# save vocab
with open(f"{NEW_MODEL_NAME}/tfjs/vocab.json", "w") as f:
    json.dump(vocab, f)

# copy tokenizer.py to tfjs folder
shutil.copy("tokenizer.py", f"{NEW_MODEL_NAME}/tfjs/tokenizer.py")

Writing weight file tmp/model2-test/tfjs\model.json...


'tmp/model2-test/tfjs/tokenizer.py'

In [17]:
raise Exception("stop here")

Exception: stop here

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(f"{NEW_MODEL_NAME}/saved_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_quant_model = converter.convert()

In [ ]:
# make folder
os.makedirs(f"{NEW_MODEL_NAME}/tflite", exist_ok=True)

# save tflite model
with open(f"{NEW_MODEL_NAME}/tflite/model.tflite", "wb") as f:
    f.write(tflite_quant_model)

# save vocab
with open(f"{NEW_MODEL_NAME}/tflite/vocab.json", "w") as f:
    json.dump(vocab, f)

# copy tokenizer.py to tflite folder
shutil.copy("tokenizer.py", f"{NEW_MODEL_NAME}/tflite/tokenizer.py")

'tmp/model2-test/tflite/tokenizer.py'